### Purposes:
read records from RIS format paper records;
need to automatically get all the indices of each item of paper information;

### problem:
If use dict to collect each paper record, so that one dict contains only single record, after file reading, it is a little bit difficult to convert the each dict to pandas.DataFrame; so I have use `dict.values()` to convert each dict into list in order to convert to DataFrame to save file;
#### Initially,
the reason that I used dict to collect each paper record is that: 
- it seemed better to leave a blank when a specific item of inforamtion is absent in the record, so I think I can use key::value pair to set the blank;
#### However, the list can work the same:
- just need to initiate a list that has the same length of the scale, and 
- for reading each item, use `scale.index(line[:2])` to index the position where a specific item of paper information should be insert to;

### Summary:
I just wander in a user-customized function, it seems `list` can do every when you need a sequence; so in what situation or condition, it is better to use `dict` or `set` ? Well, I just think the usage of these two kinds seems to be largely overlapped by `list`.

In [ ]:
def c_or_a():
    print('Please enter c to confirm, a to abandan.')
    while True:
        dec = input('Please enter c to confirm, a to abandan')
        if dec == 'c':
            return False
        elif dec == 'a':
            return True
        else:
            print('Unrecognized input, please enter again.')

def dict_gen(keys):
    dct = {}
    for key in keys:
        dct[key] = []
    yield dct

def read_scale(f):
	'''
	Attention: The line breaks of file should be '\n'

	Usage: read a plain text file of paper records and return the scale of its indices for reading contents

	: param f: the fileIO of the record file

	Suggestion: when TY is not equal to ER: check for record file to complete the 'TY -> ER' frame
	'''
	f.seek(0)
	# count the frequency
	dct = {}
	for item in f.readlines():
		if not item == '\n': # it is possible a file do not contain any blank row
			dct[item[:2]] = dct.get(item[:2], 0) + 1
		else:
			dct['NullStr'] = dct.get('NullStr', 0) + 1
	# check for the equality of each indices charactors
	print(f"Please check for equality of TY and ER, and check other index's frequency:")
	for key in dct:
		print(f'{key} : {dct[key]}')
	if c_or_a():
		return
	return tuple(dct.keys())

def start_reading(f, scale):
	'''
	it must be ensured that each record in file(f) starts with 'TY' and ends with 'ER', not of each can be missing;

	debug log:
		'count-not-equl' problem: check for the equality of two indices (TY & ER) in read_scale()
	'''
	f.seek(0)
	lst_res = []
	for line in f:
		if line[:2] == 'TY':
			for itr in dict_gen(keys=scale):
				imdct = itr
			imdct['TY'].append(line[6:-1])
		elif line[:2] == 'ER':
			lst_res.append(imdct)
		elif line[:2] in scale:
			imdct[line[:2]].append(line[6:-1])
	return lst_res

def convert_dct_to_list(lst_res:list):
    lst_new = []
    for i in range(len(lst_res)):
        for key in lst_res[i]:
            lst_res[i][key] = '; '.join(lst_res[i][key])
        lst_new.append(list(lst_res[i].values()))
    return lst_new

# f = open('Data\My Collection.md', 'rt', encoding='utf_8')
# scale = read_scale(f)
# lst_of_dct = start_reading(f, scale)
# lst_for_output = convert_dct_to_list(lst_of_dct)

In [ ]:
import pandas as pd
# dtfm = pd.DataFrame(lst_for_output, columns=scale)
# dtfm.to_excel('./Data/bispaperbeampoint.xlsx')